# **Age and Gender Prediction**

#Objective  
To predict a person’s **age** using a deep learning model that extracts facial features from images.  
Additionally, we aim to simultaneously predict the **gender** of the person using **multi-task learning (MTL)** to leverage shared features.

---

# Dataset  
We use the [UTKFace dataset](https://www.kaggle.com/datasets/jangedoo/utkface-new), which contains over 20,000 facial images labeled with:
- **Age** (0–116)
- **Gender** (0: Male, 1: Female)
- **Ethnicity** (not used in this task)

---

# Approach  

## Model Architecture
We use a **pretrained ResNet18** as the backbone:
- Replace the final fully connected layer with two heads:
  - **Regression head** for age prediction (1 neuron, no activation)
  - **Classification head** for gender prediction (1 neuron, passed through sigmoid in inference)

##  Multi-Task Learning (MTL)
We train the model on both tasks simultaneously:

| Task     | Type               | Loss Function         |
|----------|--------------------|------------------------|
| Age      | Regression          | Mean Squared Error (MSE) |
| Gender   | Binary Classification | BCEWithLogitsLoss      |

---

#  Problem: Loss Scale Imbalance

Age and gender losses operate on **different numerical scales**:
- Age loss (MSE) ≈ 10 to 100+
- Gender loss (BCE) < 1

Solution: Learnable Uncertainty Weighting (Kendall et al.) We apply the method from the paper:
-  "[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics"](https://arxiv.org/abs/1705.07115)
   - Access Paper > View PDF. This equation is at the bottom of page 5 in the report.

- Idea
  Each task is associated with a learnable uncertainty parameter (variance σ²), and the total loss is:
$$
\mathcal{L}_{\text{total}} = \frac{1}{2\sigma_1^2} \mathcal{L}_{\text{age}} + \log \sigma_1 + \frac{1}{2\sigma_2^2} \mathcal{L}_{\text{gender}} + \log \sigma_2
$$

- Where:

- $ \mathcal{L}_{\text{age}} $ and $ \mathcal{L}_{\text{gender}} $ are the individual losses for the age and gender tasks.
- $ \sigma_1^2 $ and $ \sigma_2^2 $ are the uncertainties (variances) for the age and gender tasks respectively. These uncertainties are learned as log-variance parameters during training, denoted $ \log \sigma_1 $ and $ \log \sigma_2 $.

This approach allows the model to automatically balance the contributions of the tasks based on their uncertainties. If one task is more uncertain, the model will give it less weight in the final total loss.

Thank you for your interest.

Best regards,

Duy

<h2>Table of Contents</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="#1-import-data">1. Import Data</a></li>
    <li><a href="#2-building-and-training">2. Building And Training</a>
        <ul>
            <li><a href="#21-building-and-training">2.Building And Training</a></li>
            <li><a href="#22-restoring-the-model-in-the-event-of-incidents">2.2 Restoring the Model in the Event of Incidents</a></li>
        </ul>
    </li>
</ul>
</div>

<hr>

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn

import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import torch.nn.functional as F

from torch.utils.data import DataLoader
import cv2

from PIL import Image
import os
import torchvision.models as models
import torch.optim as optim

from tqdm.autonotebook import tqdm

from torch.utils.tensorboard import SummaryWriter

from torchvision.models import resnet18


## **1. Import Data**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jangedoo/utkface-new")

print("Path to dataset files:", path)

In [ ]:

class ReadDataset(Dataset):
    def __init__(self, root, train = True, transform = None):
        self.path = os.path.join(root, "UTKFace" if train else "crop_part1")
        self.images, self.age_labels, self.gender_labels = [], [], []

        # Define the default transform if none provided
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),  # c,w,h -> h,w,c
            transforms.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Scale [-1, 1]
        ])

        # Read the image paths and labels
        for file_name in os.listdir(self.path):
            file_path = os.path.join(self.path, file_name)
            self.images.append(file_path)

            # Get age for label (assuming file_name format is like "age-gender-otherinfo.jpg")
            age = int(file_name.split("_")[0])
            self.age_labels.append(float(age))  # Label is the age
            self.gender_labels.append(int(file_name.split("_")[1]))  # Label is the gender


    def __len__(self):
        return len(self.age_labels)

    def __getitem__(self, index):
        # Load image using cv2 (in BGR format)
        img_path = self.images[index]
        img = cv2.imread(img_path)

        # Convert BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Convert image to PIL Image to apply transforms
        img = Image.fromarray(img)

        if self.transform:
            img = self.transform(img)  # Apply transformations

        # Get the label as a tensor
        age_label = torch.tensor(self.age_labels[index], dtype=torch.float)
        gender_label = torch.tensor(self.gender_labels[index], dtype=torch.float)

        return img, age_label, gender_label


root = path

# data augmentaion hepls models avoid overfitting
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(degrees=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# get dataset
train_dataset = ReadDataset(root, train = True, transform = transform)
test_dataset = ReadDataset(root, train = False)
train_loader = DataLoader(train_dataset, batch_size= 64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size= 64, shuffle=True)


In [ ]:
train_dataset.__len__()

In [ ]:
image,age,gender = train_dataset.__getitem__(10)
image.shape

In [ ]:
def show_batch(images, age, gender):
    index = int(np.random.random_integers(0, 64))
    # Un-normalize
    images = images[index] * 0.5 + 0.5  #  [-1, 1] → [0, 1]
    cater = ["Male","Female"]
    # Hiển thị
    plt.figure(figsize=(15, 8))
    plt.imshow(images.permute(1, 2, 0))  # [C, H, W] → [H, W, C]
    if age is not None:
        plt.title(f"Labels: {age[index]}.....{cater[int(gender[index])]}")
    plt.axis('off')
    plt.show()

In [ ]:

# get the first batch
images, age, gender = next(iter(train_loader))

# Hiển thị batch
show_batch(images, age, gender)


## **2. Building And Training**

### **2.1 Building And Training**

**Let's see ResNet18's Architecture**

In [ ]:
resnet_pretrain = resnet18(pretrained=True)
print(resnet_pretrain)

**Adapting this Model to be suitable of this Problem accordingly**

In [ ]:
class adapt_resnet18(nn.Module):
    def __init__(self):
        super().__init__()
        old_model = resnet18(pretrained=True)

        # Keep only the basic parts of ResNet-18
        self.features = nn.Sequential(
            old_model.conv1,
            old_model.bn1,
            old_model.relu,
            old_model.layer1,
            old_model.layer2,
            old_model.layer3,
            nn.AdaptiveAvgPool2d((1, 1)),  # Pooling to 1x1 size
        )

        self.log_var_age = nn.Parameter(torch.tensor(0.0))
        self.log_var_gender = nn.Parameter(torch.tensor(0.0))

        # A single dropout layer after flattening, it helps model avoid overfitting
        self.dropout = nn.Dropout(0.5)

        # Head age (regression)
        self.age_head = nn.Sequential(
            nn.Linear(256, 128),  # Input should be 256
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)  # Output: Age (float)
        )

        # Head gender (classification)
        self.gender_head = nn.Sequential(
            nn.Linear(256, 1),  #
        )

    # Multi-Task Learning Using Uncertainty to Weigh Losses
    def get_total_loss(self, age_out, age_label, gender_out, gender_label, age_criterion, gender_criterion):
      age_loss = age_criterion(age_out, age_label)
      gender_loss = gender_criterion(gender_out, gender_label)

      total_loss = (
          torch.exp(-self.log_var_age) * age_loss + self.log_var_age +
          torch.exp(-self.log_var_gender) * gender_loss + self.log_var_gender
      )
      return total_loss

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # Flatten from (B, 512, 1, 1) to (B, 512)
        x = self.dropout(x)      # Apply dropout to the entire layer

        age_pred = self.age_head(x)
        gender_pred = self.gender_head(x)

        return age_pred, gender_pred


In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model = adapt_resnet18()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-5)

# Loss function
age_criterion = nn.MSELoss()                       # Regression -
gender_criterion = nn.BCEWithLogitsLoss()          # Classification - (logit + sigmoid tích hợp)

In [ ]:
# create Directory to save model
if not os.path.exists('Models'):
    os.mkdir('Models')
    print("Directory 'Models' has been created.")
else:
    print("Directory 'Models' already exists.")

In [ ]:
# save best model including number of epoch, state_dict, optimizer and loss
def save_checkpoint(state, filename="checkpoint.pth.tar"):
    torch.save(state, filename)

In [ ]:
# saving the best model when training
def train_validate_and_save_best_mode(model, age_criterion,gender_criterion, optimizer, epochs, file_name):

    train_loss_list, test_loss_list = [], []
    model.to(device)
    best_loss = float('inf')
    for epoch in range(1, epochs+1):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        alpha = 1.0  # Hệ số cho age_loss
        beta = 1.0   # Hệ số cho gender_loss
        ###################
        # train the model #
        ###################
        model.train()
        train_loader_bar = tqdm(train_loader, desc = "Trainning", leave=False) # illustrate process

        for batch_idx, (data, age_labels, gender_labels) in enumerate(train_loader_bar):

            # move tensors to GPU if CUDA is available
            data = data.to(device)

            age_labels = age_labels.view(-1, 1)
            gender_labels = gender_labels.view(-1, 1)
            age_labels = age_labels.to(device)
            gender_labels = gender_labels.to(device)

            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            age_out, gender_out = model(data)
            total_loss = model.get_total_loss(age_out, age_labels, gender_out, gender_labels, age_criterion, gender_criterion)

            # Backpropagation và cập nhật optimizer
            total_loss.backward()
            optimizer.step()
            # update training loss
            train_loss += total_loss.item()*data.size(0)

            train_loader_bar.set_postfix(loss = total_loss.item())

        ######################
        # validate the model #
        ######################
        model.eval()

        test_loader_bar = tqdm(test_loader, desc = "Validate", leave=False)

        for batch_idx, (data, age_labels, gender_labels) in enumerate(test_loader_bar):
            with torch.no_grad():
              # move tensors to GPU if CUDA is available
              data = data.to(device)

              age_labels = age_labels.view(-1, 1)
              gender_labels = gender_labels.view(-1, 1)

              age_labels = age_labels.to(device)
              gender_labels = gender_labels.to(device)
              age_out, gender_out = model(data)
              total_loss = model.get_total_loss(age_out, age_labels, gender_out, gender_labels, age_criterion, gender_criterion)
              # update training loss
              valid_loss += total_loss.item()*data.size(0)

              test_loader_bar.set_postfix(loss = total_loss.item())

        # calculate average losses
        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(test_loader.sampler)

        if valid_loss < best_loss:
            best_loss = valid_loss
            save_checkpoint({
                'epoch': epoch + 1 ,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': valid_loss
            }, filename=("Models/{}.pth.tar".format(file_name)))

        # print training/validation statistics
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))

        train_loss_list.append(train_loss)
        test_loss_list.append(valid_loss)

    return train_loss_list, test_loss_list

In [ ]:
## visualization for training and validating
def evaluate(nb_epoch, train_loss_list, test_loss_list):
    epoch_list = [i for i in range(1, nb_epoch+1)]
    plt.plot(epoch_list,train_loss_list, marker = "o" , color = "blue")
    plt.plot(epoch_list,test_loss_list, marker = "o" , color = "red")

    plt.show()


In [ ]:
## training
epochs = 20
train_loss_list_v1, test_loss_list_v1 = train_validate_and_save_best_mode(model, age_criterion,gender_criterion, optimizer, epochs, "age_gender_predicting_model")
evaluate(epochs,train_loss_list_v1,test_loss_list_v1)

### **2.2 Restoring the Model in the Event of Incidents**

In [ ]:
def load_checkpoint(filepath, model, optimizer=None):
    # Load checkpoint từ file
    checkpoint = torch.load(filepath)

    # Load trạng thái của model
    model.load_state_dict(checkpoint['state_dict'])

    # Nếu có optimizer, load trạng thái của optimizer
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer'])

    # Trả lại epoch hiện tại và giá trị loss
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    # In thông tin để xác nhận
    print(f"Checkpoint loaded: epoch {epoch}, loss {loss:.4f}")

    return epoch, loss


In [ ]:
# /age_gender_predicting_model.pth.tar


In [ ]:
# Load checkpoint from file
max_epochs = 20
model_retrain = adapt_resnet18()
start_epoch, previous_loss = load_checkpoint("/age_gender_predicting_model.pth.tar", model_retrain, optimizer)

In [ ]:
train_loss_list, test_loss_list = train_validate_and_save_best_mode(model_retrain,age_criterion,gender_criterion, optimizer, max_epochs - start_epoch + 10, "age_gender_predicting_model")
evaluate(epochs,train_loss_list,test_loss_list)